In [ ]:
import os.path
import re

import pandas as pd
import numpy as np

from scipy.cluster.vq import kmeans, kmeans2
from sklearn.cluster import KMeans

Load the data and preprocess it.

In [ ]:
FILEPATH = os.path.abspath("../data/2023-03-25_22hr_29min_ptwr_data.tsv")
df = pd.read_csv(FILEPATH, sep="\t")

MAKE = "make"
MODEL = "model"
SPECS = "specs"

CLEAN_MAKE = re.compile(r"((/make/)|(\-power\-to\-weight\-ratio\-stats))")
CLEAN_MODEL = re.compile(r"(/model/)")
CLEAN_SPECS = re.compile(r"(\\t|\\n)")

df[MAKE] = df[MAKE].apply(lambda x: CLEAN_MAKE.sub("", x))
df[MODEL] = df[MODEL].apply(lambda x: CLEAN_MODEL.sub("", x))
df[SPECS] = df[SPECS].apply(lambda x: CLEAN_SPECS.sub("", x))

SEPARATOR = ":"
SPECS1 = "specs1"
df[SPECS1] = df[SPECS].apply(lambda x: float(x.split(SEPARATOR)[-1]))
df[SPECS] = df[SPECS].apply(lambda x: SEPARATOR.join((x.split(SEPARATOR)[:-1])))

print(df.describe())
print(df.head())

# Introduction

I am going to rely upon classifiers which have discriminants calculated from the mean of the training data set and also what is calculated using the scipy library's kmeans2 clustering algorithm. I will use both of these techniques to model power-to-weight ratios per model year and assess the effectiveness when the data is partitioned in this way.

# Attempt 1 Classification: Mean Discriminator